In [41]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from pathlib import Path
from tratamento import trata_acomph, a0_a1, tipo3

In [42]:
def get_prod():
    loc = Path('../ex_csv/produtibilidades/prod.csv')
    prod = pd.read_csv(loc, index_col=0)
    for i,row in prod.head(154).iterrows():
        row['prod'] = row['prod'].replace(",",".")
        row['prod'] = float(row['prod'])
    return prod

In [43]:
def importa_arquivos():
    trata_acomph.ex_final()
    acomph = trata_acomph.get_csv()
    produtibilidade = get_prod()
    produtibilidade.drop_duplicates(inplace=True)
    a0, a1 = a0_a1.get_csv()
    local = Path('../ex_csv')
    local_post = local / 'postos.csv'
    postos = pd.read_csv(local_post, index_col=0)
    return acomph, a0, a1, postos, produtibilidade

In [44]:
def get_datas():
    meses = []
    datas = []
    for i in range(1,31):
        datas = [str(date.today()-timedelta(days=i))] + datas
        meses = [str((date.today()-timedelta(days=i)).month)] + meses
    return meses, datas

In [45]:
def trata_vazoes_base(acomph, a1):
    vazoes_base = acomph.loc[a1.iloc[:,0], :]
    vazoes_base.reset_index(inplace=True)
    base = a1.reset_index()
    indice = base.loc[:,'posto']
    vazoes_base.insert(0, "ind", indice, allow_duplicates=True)
    vazoes_base.drop('posto', axis=1, inplace=True)
    vazoes_base.rename(columns={'ind':'posto'}, inplace=True)
    vazoes_base.set_index('posto', inplace=True)
    vazoes_base.dropna(inplace=True)
    vazoes_base.drop_duplicates(inplace=True)
    return vazoes_base

In [46]:
def atualiza_acomph(acomph, a0, a1, postos):
    acomph = regressao_tipo_1(acomph,a0,a1)

In [47]:
def cria_ena():
    acomph, a0, a1, postos, produtibilidades = importa_arquivos()
    ind = acomph.head(155).index
    col = (acomph.T).head(30).index
    ena = pd.DataFrame(0,index = ind, columns = col)
    return ena

In [48]:
def regressao_tipo_1(acomph, a0, a1):
    meses, datas = get_datas()
    vazoes_base = trata_vazoes_base(acomph, a1)
    display(vazoes_base)
    for i in range(30):
        A0 = a0.loc[:,meses[i]]
        display(A0)
        A1 = a1.loc[:,meses[i]]
        display(A1)
        X = vazoes_base.iloc[:,i]
        display(X)
        Y = A0 + (A1*X)
        Y.dropna(inplace=True)
        Y.rename(datas[i],inplace=True)
        Y.to_csv('../ex_csv/'+str(i)+'y.csv')
    return acomph

In [49]:
def regressao_tipo_3(acomph):
    postos_tipo3 = [tipo3.posto_126(), tipo3.posto_127(), tipo3.posto_131(), tipo3.posto_132(),
                    tipo3.posto_176(), tipo3.posto_285(), tipo3.posto_292(), tipo3.posto_298(), 
                    tipo3.posto_299(), tipo3.posto_303(), tipo3.posto_304(), tipo3.posto_306(),
                    tipo3.posto_315(), tipo3.posto_316(), tipo3.posto_317(), tipo3.posto_318(),
                    tipo3.posto_37(), tipo3.posto_38(), tipo3.posto_39(), tipo3.posto_40(),
                    tipo3.posto_42(), tipo3.posto_43(), tipo3.posto_45(), tipo3.posto_46(),
                    tipo3.posto_66(), tipo3.posto_75()]
    print(postos_tipo3)
    acomph = pd.concat([acomph, postos_tipo3])
    return acomph

In [50]:
a,b,c,d,e = importa_arquivos()
atualiza_acomph(a,b,c,d)

,2019-10-05,2019-10-06,2019-10-07,2019-10-08,2019-10-09,2019-10-10,2019-10-11,2019-10-12,2019-10-13,2019-10-14,...,2019-10-25,2019-10-26,2019-10-27,2019-10-28,2019-10-29,2019-10-30,2019-10-31,2019-11-01,2019-11-02,2019-11-03
posto,,,,,,,,,,,,,,,,,,,,,
2,33.663404,32.843718,32.940487,38.733755,52.926055,59.684476,53.005693,50.383941,45.228366,40.019959,...,32.803813,35.048803,37.621017,35.945975,36.863176,37.138094,42.489103,49.111555,48.732832,46.248360
7,185.997892,175.051666,167.434987,163.200579,164.059402,183.174192,235.470853,295.811970,298.519527,260.951828,...,150.004989,152.062756,155.822446,160.627569,163.501844,174.284856,187.887970,194.275845,195.060994,197.563968
15,21.812326,21.912215,21.536272,23.137851,22.647315,24.511009,26.249833,23.384943,22.641163,20.474079,...,19.277006,19.650939,23.585978,24.783738,24.902330,21.623880,21.567669,21.650926,22.011767,21.225861
22,77.471327,78.339111,76.522533,81.765557,85.625583,94.677249,91.740171,87.798205,84.548226,81.969999,...,74.612826,76.567948,76.951209,75.727944,73.742250,73.353962,75.239566,87.017783,91.618725,94.362532
206,77.976954,72.775405,75.043130,79.724719,81.409398,83.234281,85.408959,102.335323,113.885763,86.877141,...,70.905869,69.834482,70.769366,72.468566,71.105250,74.377902,82.057429,83.671774,85.148399,87.557408
23,19.823319,20.139186,19.909995,20.278021,20.072914,20.710585,20.800256,19.805346,19.559289,18.641059,...,22.458931,24.569559,25.720683,25.039736,24.026219,21.736136,20.775918,22.086341,23.204183,24.425314
32,296.653498,288.115571,285.615828,288.835788,304.081092,322.283696,340.076911,344.672702,343.060545,336.707252,...,285.202813,296.491129,307.571946,309.748706,306.645092,303.736970,304.255937,317.352709,337.654172,352.014470
248,70.401294,76.280005,84.089204,91.130931,96.043082,96.219430,92.308090,86.205022,80.873818,77.114404,...,79.824576,75.749436,73.846608,74.059946,74.059701,73.341252,73.851229,76.475121,79.388480,80.766914
241,96.543472,99.501071,102.663622,106.061535,108.701206,110.823661,111.103405,110.401233,108.971706,105.772847,...,105.251539,103.392959,101.246694,100.723242,101.023859,102.499108,105.017399,106.634695,107.321628,108.048979


posto
2      0.0
7      0.0
8      0.0
9      0.0
10     0.0
      ... 
228    0.0
230    0.0
204    0.0
297    0.0
55     0.0
Name: 10, Length: 68, dtype: float64

posto
2      1.000
7      1.109
8      1.131
9      1.141
10     1.165
       ...  
228    0.591
230    1.009
204    0.996
297    1.011
55     0.660
Name: 10, Length: 68, dtype: float64

posto
2        33.663404
7       185.997892
15       21.812326
22       77.471327
206      77.976954
23       19.823319
32      296.653498
248      70.401294
241      96.543472
116      17.607340
48       58.576590
62      479.836724
89       77.898160
217     355.450282
103     362.580435
76      244.032482
72       18.442728
78      356.878858
252      44.885004
110     615.827802
97      247.622633
303       6.694293
123      94.070161
129     193.942454
122      44.340927
198      10.318181
263      25.486333
141     187.717234
183      13.209568
191      95.826468
273     221.088960
155      98.978300
285    3641.698308
227     593.960652
204     343.255306
55       78.844374
Name: 2019-10-05, dtype: float64

/home/mazucanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  from ipykernel import kernelapp as app


...

posto
2      1.000
7      1.103
8      1.123
9      1.130
10     1.152
       ...  
228    0.519
230    1.010
204    1.000
297    1.010
55     0.660
Name: 11, Length: 68, dtype: float64

posto
2        48.732832
7       195.060994
15       22.011767
22       91.618725
206      85.148399
23       23.204183
32      337.654172
248      79.388480
241     107.321628
116      15.293800
48       54.148911
62      459.197898
89      333.336223
217    1694.530978
103     728.984341
76      272.925881
72       23.400506
78      352.661342
252      42.133759
110     663.724693
97      403.343593
303       9.750303
123     124.989773
129     321.056115
122      28.398465
198      15.828919
263      53.572774
141     227.518487
183      32.540704
191     110.920204
273     198.290835
155     105.795239
285    6500.876255
227     824.782213
204     279.568699
55       80.859304
Name: 2019-11-02, dtype: float64

posto
2      0.0
7      0.0
8      0.0
9      0.0
10     0.0
      ... 
228    0.0
230    0.0
204    0.0
297    0.0
55     0.0
Name: 11, Length: 68, dtype: float64

posto
2      1.000
7      1.103
8      1.123
9      1.130
10     1.152
       ...  
228    0.519
230    1.010
204    1.000
297    1.010
55     0.660
Name: 11, Length: 68, dtype: float64

posto
2        46.248360
7       197.563968
15       21.225861
22       94.362532
206      87.557408
23       24.425314
32      352.014470
248      80.766914
241     108.048979
116      15.293633
48       52.646018
62      451.263640
89      296.315266
217    1537.169328
103     739.642514
76      271.768576
72       20.274463
78      341.588037
252      47.132259
110     391.069302
97      306.122678
303       8.580532
123     131.890239
129     280.072303
122      26.373240
198      14.712478
263      55.503059
141     230.422869
183      31.678793
191     120.582287
273     192.946456
155     111.888251
285    6343.308923
227     841.761752
204     277.921594
55       70.971723
Name: 2019-11-03, dtype: float64